In [442]:
from operator import index
import random
import re
import nltk
import pandas as pd
from csv import reader
from nltk.tokenize import word_tokenize
import csv
'''
sorry, didn't have enough time to organize into classes and functions :(
    
Step 2) Train a Naive Bayes classifier (Program 1)

2.a) Read in training set from the train.csv file 

2.b) Calculate the prior probabilities for both classes using the training set

2.b) Calculate the conditional probability of each unique word in the training set given a class. Compute these conditional probabilities for each class. 

2.c) Output the learnt classification model (all the above probabilities) to a file named model.csv    


HAVE TIME TO COMMENT THIS PART
TRAINING PROGRAM:
create a hashmap: key = class value = documents
we will have 2 class
tokenize, normalize,find vreq, unique value of class
find prior prob P(0), P(1)
find likelihood conditional prob for each word

output the probablity to a csv file: model.csv
'''
#pos = positive class 
#neg = negative class


# read csv file as a list of lists
with open('train.csv','r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Pass reader object to list() to get a list of lists
    list_of_rows = list(csv_reader)

#print(list_of_rows)
#all the vocabs from csv file
vocablist = ""
pos_vocab = "" #positive class vocabs
neg_vocab = "" #negative class vocabs

pos_doc = 0 #counts total # of positive document
neg_doc = 0 #counts total # of negative document

#splitting positive and negative class vocabs and storing into seperate string
for words in list_of_rows:
    vocablist = vocablist + words[0] + " "
    if words[1] == "positive":
        pos_doc += 1
        pos_vocab = pos_vocab + words[0] + " "
    else:
        neg_doc += 1
        neg_vocab = neg_vocab + words[0] + " "

#Calculate the prior probabilities for both classes using the training set
#total # documents in csv file
total_doc = len(list_of_rows)
priors_prob_pos = pos_doc / total_doc
priors_prob_neg = neg_doc / total_doc

#print(priors_prob_pos)
#print(priors_prob_neg)


#function to tokenize and normalize the vocablist
def vocab_tokenize(valuec):
        normalize_corpus = valuec.lower()
        tokenize = word_tokenize(normalize_corpus)
        tokenize=[word.lower() for word in tokenize if word.isalpha()]
        return tokenize
  
#counting frequency of all words in the corpus
#returns dict of word:freq
def vocab_builder(vocabs):
        vocab = {}
        for x in vocabs:
            if vocab.get(x) == None:
                vocab[x] = 1
            else:
                vocab[x] = vocab.get(x)+1
        return vocab
    
#all the vocabs from csv file
vocab_token = vocab_tokenize(vocablist)
vocab_freq = vocab_builder(vocab_token)
total_vocab = len(vocab_freq)

#vocab from positive class label
pos_class_token = vocab_tokenize(pos_vocab)
pos_class_freq = vocab_builder(pos_class_token)
toal_word_pos = len(pos_class_token)

#vocab from negative class label
neg_class_token = vocab_tokenize(neg_vocab)
neg_class_freq = vocab_builder(neg_class_token)
total_word_neg = len(neg_class_token)

#Calculate the conditional probability of each unique word in the training set given a class. Compute these conditional probabilities for each class. 

#conditional probability of positive class
pos_cond_probs = {} #hashmap of word:cond prob of pos class
for word in pos_class_freq:
    pos_cond_probs[word] = (pos_class_freq[word] + 1) / (toal_word_pos + total_vocab) #using conditional probability formula and laplace smoothing of 1
#print(f"pos class prob: {pos_cond_probs}")


#conditional probability of negative class
neg_cond_probs = {} #hashmap of word:cond prob of neg class
for word in neg_class_freq:
    neg_cond_probs[word] = (neg_class_freq[word]+1) / (total_word_neg + total_vocab)
    
#for debug
#print(f"negative class prob: {neg_cond_probs}")
#print(f"test: {neg_class_freq['difficult']}  count(c): {total_word_neg} |V|: {total_vocab}")


#Output the learnt classification model (all the above probabilities) to a file named model.csv
#opening and writing to model.csv file
with open('model.csv','w',newline='') as model:
    writer = csv.writer(model)
    
    #writing prior probablity to model.csv
    writer.writerow(['Prior probablity'])
    writer.writerow(['class','probablity'])
    writer.writerow(["negative", priors_prob_neg])
    writer.writerow(["positive", priors_prob_pos])
    
    #writing header labels
    writer.writerow(['word','class','probablity'])
    
    #writing negative class cond probablity to model.csv
    for word in neg_cond_probs:
        temp_neg = [word, "negative", neg_cond_probs[word]]
        writer.writerow(temp_neg)
        
        
    #writing positive class cond probablity to model.csv
    for word in pos_cond_probs:
        temp_pos = [word, "positive", pos_cond_probs[word]]
        writer.writerow(temp_pos)




In [444]:
from operator import index
import random
import re
import nltk
import pandas as pd
import math
import csv
from csv import reader

"""
sorry about messy code, couldn't 
Step 3) Use the learnt NB classifier (Program 2)

3.a) Read in the model.csv file and the test.csv files 

3.b) Use the classification model to predict class label for each document in the test set

3.c) Output each of the test document, its predicted class label, and its actual class label to a file named test_predictions.csv
"""



# Read in the model.csv file and the test.csv files
#reading test and model file and storing it as list
with open('model.csv','r') as model_obj:
    # pass the file object to reader() to get the reader object
    model_reader = reader(model_obj)
    # Pass reader object to list() to get a list of lists
    model_list = list(model_reader)
    
with open('test.csv','r') as test_obj:
    # pass the file object to reader() to get the reader object
    test_reader = reader(test_obj)
    # Pass reader object to list() to get a list of lists
    test_list = list(test_reader)
    

#Use the classification model to predict class label for each document in the test set

#prior probablity of each class
pos_prior_prob = model_list[3][1]
neg_prior_prob = model_list[2][1]
#print(f"pos PP: {pos_prior_prob}")
#print(f"neg PP: {neg_prior_prob}")

positive_class = {} #hashmap of word:cond prob of positive class
negative_class ={} #hashmap of word:cond prob of negative class

#looping through list of model.csv to differenciate positive and negative class
for x in range(5, len(model_list)):
    if(model_list[x][1] == "positive"):
        positive_class[model_list[x][0]] = model_list[x][2]
    else:
        negative_class[model_list[x][0]] = model_list[x][2]

label_list = [] #holds list of test and predicted labels

for x in test_list: #looping from each document of test.csv
    pos_prob = 0;
    neg_prob = 0;
    temp = x[0].lower()
   # print(x[0])
    for word in temp.split(): #looping through each word of document of test.csv
        #print(word)
        #checking if word is in our label class
        if word in positive_class: 
            pos_prob += math.log(float(positive_class[word])) #gets word probablity from positive class and calculate. using  log to avoid smaller numbers
        if word in negative_class:
            neg_prob += math.log(float(negative_class[word])) #gets word probablity from negative class and calculate. using  log to avoid smaller numbers
    pos_prob += math.log(float(pos_prior_prob)) #adding positive class prior probabilities ; part of formula
    neg_prob += math.log(float(neg_prior_prob)) #adding negative class prior probabilities ; part of formula
    
    # ['test document','predicted class','actual class label']) : example format to store in file
    #comparing the probabilities of positve vs negative, since we are using log, numbers are flipped so smaller number is higher prob
    if pos_prob < neg_prob:
        label_list.append([temp,'positive',x[1]])
    else:
        label_list.append([temp,'negative',x[1]])

#print(label_list)

#Output each of the test document, its predicted class label, and its actual class label to a file named test_predictions.csv
with open('test_predictions.csv','w',newline='') as predict:
    writer = csv.writer(predict)
    #writing to test_preditcion file
    writer.writerow(['test document','predicted class','actual class label'])
    for word in label_list:
        temp_row = [word[0], word[1], word[2]]
        writer.writerow(temp_row)


